Notebook to proceed with the following steps once the yolo model has been trained. Will run manually here sequences of the classify.py and testing.py scripts. 

Ideally, the runing of this section can be done by calling modes that run pipelines out of Charlie's code. Let's see how much of those automatisations can be used here. 
Here is his comment from the testing.py file:

Utility functions for training/validation pipeline.  
Includes: 

    - prepare: Prepare the images for segmentation
    - segment: Segment the images
    - run_detection: Run the YoloV5 detection
    - backwards_annotation: Generate labelme style annotations from the classifications
    - compare_detections_to_ground_truth: Match up labels and detections, compare them, and save the results
    - confusion_matrix: Summarize the results of the comparison

Load first all the configs and required packs:

In [1]:

import os
import shutil
import yaml 
import argparse
import os.path as path
import scipy.cluster
import scipy.spatial
import json
import sys
import subprocess

import numpy as np
import pandas as pd
import scipy
import random
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

from tqdm import tqdm
import torch
import stat
import shutil
from datetime import datetime

from counting_boats.boat_utils.config import cfg
from counting_boats.boat_utils import image_cutting_support as ics
from counting_boats.boat_utils import heatmap as hm

import counting_boats.boat_utils.classifier
# cluster, process_clusters, read_classifications, pixel2latlong

# Add the project root to sys.path (adjust as needed)
sys.path.append(os.path.abspath("counting_waterholes"))


c:\Users\fossatia\AppData\Local\miniconda3\envs\Boats\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  v7.0-394-g86fd1ab2 Python-3.10.16 torch-1.12.1+cu113 CUDA:0 (GeForce GTX 1080, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Then we can select manually which function we want to run. 
First, from the testing aoi tif file, we need to prepare the padded png image using the testing.prepare():

In [ ]:
# Run preparation:
import counting_boats.boat_utils.testing


counting_boats.boat_utils.testing.prepare("counting_waterholes/testing", "config_test_GPU.yaml")

Then I need to use the created png to label it with labelme. This will allow us to compare my annotation to the detection of the trained model i.e. test the model. 

Once the manual annotation is done, we can apply the segmentation used from the testing.segment():

In [2]:
#run segmentation without spliting 80% of the images for validation!
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.segment(r"D:/Waterholes_project/counting_waterholes/testing", "config_test_Drive.yaml")

Cropping Image: D:/Waterholes_project/counting_waterholes/testing\./pngs\20240204_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
0% of images without labels will be removed


Saving Segments: 100%|██████████| 15933/15933 [05:33<00:00, 47.77it/s] 


Skipped 728 images
Empty 0 images
Cropping Image: D:/Waterholes_project/counting_waterholes/testing\./pngs\20240324_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
0% of images without labels will be removed


Saving Segments: 100%|██████████| 15933/15933 [08:10<00:00, 32.46it/s] 


Skipped 1676 images
Empty 0 images
Cropping Image: D:/Waterholes_project/counting_waterholes/testing\./pngs\20240415_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
0% of images without labels will be removed


Saving Segments: 100%|██████████| 15933/15933 [11:48<00:00, 22.49it/s] 


Skipped 790 images
Empty 0 images
Segregating by day...
01_01_2024
04_02_2024
24_03_2024
15_04_2024
Segregating by image...
Segregating by image...
Segregating by image...
Segregating by image...
Segregating by day...
01_01_2024
04_02_2024
24_03_2024
15_04_2024
Segregating by image...
Segregating by image...
Segregating by image...
Segregating by image...


Using those segmented labelled images, we can run the detection of waterholes using the trained model, and compare my label with the detection of the model. 

Debugging section to recognise and work well with the GPU:

Note to user: Need to update the torchvision to match the cuda (GPU) version. using the 'nvidia-smi' command, you get the cuda version (my case: 11) so I need to get a version of torch and torchaudio with to 11.xx. Need to 'pip uninstall torch torchvision', and then install the correct version, in my case: 'pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113'  
Other version to be found on this website: https://pytorch.org/get-started/previous-versions/

The testing.segment function and run_detection work to use the segmented images folders grouped per date. left as it is for now but just something to bear in mind!  

In [ ]:
#check of the GPU found or not?
torch.cuda.is_available()

True

In [ ]:
#run detection on my testing 
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.run_detection(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")

Copy of the order command that is run run to yolo cmd  by the function run_detectionpage. Just in order to debug and be informative. Do not run.

In [ ]:
python C:/Users/fossatia/Documents/Waterholes_project/yolov5/detect.py --imgsz 416 --save-txt --save-conf --weights C:/Users/fossatia/Documents/Waterholes_project/yolov5/runs/train/exp3/weights/best.pt --source D:\\Waterholes_project\\counting_waterholes\\testing\\segmented_images\\04_02_2024\\20240204_mimal_test --device cuda:0 --nosave --conf-thres 0.15

Use the detection output of the model on my testing images to produce labelme style annotation using the backwards_annotation():  

In [3]:
#run the annotation of the images using the detection of the model:
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.backwards_annotation_AF(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")

Then finally, compare the detected WH with my labeled WH:

In [4]:
#comparison of my labels with the detected WH:
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.compare_detections_to_ground_truth(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")

Label directory D:/Waterholes_project/counting_waterholes/testing_v3\./labels\05_02_2025\050225_NT does not exist, skipping image...
raw_images folder D:/Waterholes_project/counting_waterholes/testing_v3\./raw_images
Could not parse date from 050225_NT.csv


Create the confusion matrix which summarises the results:

In [6]:
#create the confusion matrix
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.confusion_matrix_AF(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")

Possible to process a single images by comparing the detections and labels for a single image, used in a function but not usefull by hand. 

In [ ]:
# #create the confusion matrix
# import counting_boats.boat_utils.testing

# counting_boats.boat_utils.testing.process_image_AF(r"D:\Waterholes_project\counting_waterholes\testing_v3\classifications", r"D:/Waterholes_project/counting_waterholes/testing_v3/labels", "config_test_Drive.yaml")

Compare the counts one by one:

In [2]:
#comparison labelled vs detected WH:
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.waterholes_count_compare(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")

Now we want to plot the waterholes on the images. To do that, I need first to stich the training images together:

In [4]:
import os
import datetime
from PIL import Image
import counting_boats.boat_utils.stitch_PNGs

counting_boats.boat_utils.stitch_PNGs.stitch(r"D:\Waterholes_project\counting_waterholes\testing_v3\stitching")

Max x: 92, Max y: 60
Saved stitched image to D:\Waterholes_project\counting_waterholes\testing_v3\stitching\stitched.png


Now I want to plot the waterholes, but the function is made for boats so I need to modify it to work on WH. WIP

In [ ]:
#plot WH using that stiched image:
import counting_boats.boat_utils.testing

counting_boats.boat_utils.testing.plot_waterholes(r"D:/Waterholes_project/counting_waterholes/testing_v3", "config_test_Drive.yaml")